In [ ]:
import geopandas as gpd
import pandas as pd

# OneBox by Allegro - Finding the greenest area

### Task 1

Load dataset with *OneBox* parcel machines (`data/oneboxes.json`) and convert *lat/lon* into a geometry column. Assign it to `one_boxes_gdf` variable


In [ ]:
data_path = '../../data/oneboxes.json'

### BEGIN SOLUTION
one_boxes_raw = pd.read_json(data_path)
one_boxes_gdf = gpd.GeoDataFrame(
    one_boxes_raw,
    geometry=gpd.GeoSeries.from_xy(one_boxes_raw["lon"], one_boxes_raw["lat"]),
    crs="EPSG:4326",
)
### END SOLUTION

In [ ]:
one_boxes_gdf.head()

In [ ]:
len(one_boxes_gdf)

In [ ]:
one_boxes_gdf.explore()

### Task 2
Clip the GeoDataFrame to contain only *OneBoxes* placed in Warsaw.

Get the area of Warsaw from the OpenStreetMap.
You can use `geocode_to_region_gdf` function from `srai.regionalizers` or `geocode_to_gdf` from `osmnx` library.

Clip the `one_boxes_gdf` using `.clip(other_geometry)` function from the GeoPandas. Assign it to `warsaw_one_boxes` variable.

In [ ]:
osm_prompt = 'Warsaw, PL'

In [ ]:
### BEGIN SOLUTION
from srai.regionalizers import geocode_to_region_gdf

warsaw_region = geocode_to_region_gdf(osm_prompt)
warsaw_one_boxes = one_boxes_gdf.clip(warsaw_region)
### END SOLUTION

In [ ]:
len(warsaw_one_boxes)

Plotting a map of OneBoxes using PyDeck and custom icons

In [ ]:
import pydeck as pdk

icon_data = {
    "url": '../../data/onebox_circle.png',
    "width": 128,
    "height": 128,
    "anchorY": 128,
}

warsaw_one_boxes["icon_data"] = [icon_data for _ in warsaw_one_boxes.index]

view_state = pdk.data_utils.compute_view(warsaw_one_boxes[["lon", "lat"]], 0.1)

icon_layer = pdk.Layer(
    type="IconLayer",
    data=warsaw_one_boxes,
    get_icon="icon_data",
    get_size=16,
    size_scale=2,
    get_position=["lon", "lat"],
    pickable=True,
)

pdk.Deck(
    layers=[icon_layer],
    map_style="road",
    initial_view_state=view_state,
    tooltip={"text": "{id}: {street}"}
)

### Task 3
Create buffers with radius 500m around OneBoxes in Warsaw.

Use the `.buffer(radius)` function from GeoPandas.
Remember not to use the standard **WGS84** CRS. Before the operation change the CRS using `to_crs` function and set it to **`EPSG:2180`** which is the Poland compatible CRS with units in meters. After buffering, reproject it back to WGS84 using `EPSG:4326` CRS.

Create a new GeoDataFrame using data from `warsaw_one_boxes` variable and use created buffers as a `geometry` column.

In [ ]:
### BEGIN SOLUTION
buffers_geometry = warsaw_one_boxes["geometry"].to_crs('EPSG:2180').buffer(500).to_crs('EPSG:4326')
warsaw_one_boxes_with_buffers = gpd.GeoDataFrame(
    data=warsaw_one_boxes,
    geometry=buffers_geometry
)
### END SOLUTION

In [ ]:
warsaw_one_boxes_with_buffers.explore(style_kwds=dict(opacity=0.5, fillOpacity=0.1))

### Task 4
Load greenery data from OpenStreetMap using `OSMPbfLoader` from `srai.loaders.osm_loaders`.
Initialize it and use function `.load()` with Warsaw geometry and OSM tags filter.

Filter is already provided and contains specific tags defining greenery objects.

Remove `Point` geometries from the collected GeoDataFrame. Use can use `geom_type` attribute of the GeoDataFrame.

Assign the GeoDataFrame to `greenery` variable.

In [ ]:
greenery_osm_tags_filter = {
    "leisure": ["garden", "park"],
    "natural": ["wood", "scrub", "heath", "grassland"],
    "landuse": ["grass", "orchard", "flowerbed", "forest", "greenfield", "meadow"],
}

In [ ]:
### BEGIN SOLUTION
from srai.loaders.osm_loaders import OSMPbfLoader

greenery = OSMPbfLoader().load(
    warsaw_region,
    greenery_osm_tags_filter
)
greenery = greenery[greenery.geom_type != 'Point']
### END SOLUTION

In [ ]:
greenery.plot(color="tab:green")

### Task 5
Intersect buffers around OneBoxes with downloaded greenery geometries. After intersection, group all intersected geometries into a single one using OneBox id.

To intersect all geometries, you can use `overlay` function from GeoPandas. It will intersectiond between each OneBox buffer and greenery object.

To join those intersections into a single geometry, use `dissolve` function from GeoPandas. It's an equivalent of `groupby` function from Pandas and automatically creates a `union` of all geometries in a group. 

Assign the final GeoDataFrame to `warsaw_one_boxes_with_greenery` variable.

In [ ]:
### BEGIN SOLUTION
warsaw_one_boxes_with_greenery = gpd.overlay(warsaw_one_boxes_with_buffers, greenery)
warsaw_one_boxes_with_greenery = warsaw_one_boxes_with_greenery.dissolve(by='id')
warsaw_one_boxes_with_greenery
### END SOLUTION

In [ ]:
warsaw_one_boxes_with_greenery.plot(color='tab:green')

### Task 6
Find the OneBox with most greenery area around it. Calculate it in a proper CRS (you can use `EPSG:2180` again).
Plot this best OneBox greenery from the buffer on a map.

In [ ]:
### BEGIN SOLUTION
warsaw_one_boxes_with_greenery['greenery_area'] = warsaw_one_boxes_with_greenery['geometry'].to_crs('EPSG:2180').area
warsaw_one_boxes_with_greenery = warsaw_one_boxes_with_greenery.sort_values(by='greenery_area', ascending=False)
warsaw_one_boxes_with_greenery.head()

best_onebox_id = warsaw_one_boxes_with_greenery.index[0]
warsaw_one_boxes_with_greenery.loc[[best_onebox_id]].explore(color="green")
### END SOLUTION

In [ ]:
m = warsaw_one_boxes_with_buffers.merge(
    warsaw_one_boxes_with_greenery["greenery_area"].reset_index(), on="id"
).explore(
    "greenery_area",
    cmap="BuGn",
    tiles="CartoDB positron",
    style_kwds=dict(opacity=0.5, fillOpacity=0.1),
)

warsaw_one_boxes.explore(m=m, color='green')